In [53]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import numpy as np
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from sparseml.pytorch.optim import ScheduledModifierManager
from sparseml.pytorch.optim import ScheduledOptimizer
from sparseml.pytorch.utils import get_prunable_layers
from sparseml.pytorch.utils import tensor_sparsity

In [ ]:
# !git clone https://github.com/neuralmagic/sparseml
# !pip install -e "sparseml[transformers]"
# !pip install sparseml
# !wget https://huggingface.co/neuralmagic/TinyLlama-1.1B-Chat-v0.4-pruned50-quant-ds/raw/main/recipe.yaml
# !pip install torchvision

In [59]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'{device} is using in this ENV.')

cuda is using in this ENV.


In [16]:
# Step 1: Load MNIST Dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=32, shuffle=False)

In [4]:
!pip install onnxruntime

In [33]:
import onnxruntime as ort

# Load the ONNX model
onnx_model_path = "./saved_model/sparse_model.onnx"
session = ort.InferenceSession(onnx_model_path)

In [63]:
# Step 8: Load Exported ONNX Model and Predict
def load_and_predict_onnx_model(onnx_path, image, label):
    import onnxruntime as ort

    # Load the ONNX model
    ort_session = ort.InferenceSession(onnx_path)

    # Prepare the input
    image_np = image
    ort_inputs = {ort_session.get_inputs()[0].name: image_np}

    # Run the model
    ort_outs = ort_session.run(None, ort_inputs)
    predicted_label = np.argmax(ort_outs[0])

    result = {
        "Predicted as" : predicted_label,
        "Ground Truth" : label
    }
    return result

In [65]:
# Use the function to predict an image from the test set
image, label = testset[2]
image = image.unsqueeze(0).to('cpu')
# load_and_predict_onnx_model(onnx_path, image.cpu().numpy())

In [66]:
# Use the function to predict
load_and_predict_onnx_model(onnx_model_path, image.cpu().numpy(), label=label)

{'Predicted as': 1, 'Ground Truth': 1}